In [1]:
## 导入需要的库
import numpy as np
import pandas as pd
import math
import os
import random
from matplotlib.pyplot import *
import matplotlib.pyplot as plt

from statsmodels.stats.weightstats import ztest

In [52]:
## 我们模型的聚类结果如下所示
lb1 = [1, 10, 13, 18, 22, 24, 25, 28, 29]
lb2 = [2, 5, 7, 8, 19, 26]
lb3 = [0, 3, 4, 6, 9, 11, 12, 14, 15, 16, 17, 20, 21, 23, 27]
label1 = []
for i in range(0,30):
    if i in lb1:
        label1.append(1)
    elif i in lb2:
        label1.append(0)
    else:
        label1.append(2)
print(label1)

[2, 1, 0, 2, 2, 0, 2, 0, 0, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 0, 2, 2, 1, 2, 1, 1, 0, 2, 1, 1]


In [42]:
## K-means聚类结果
from sklearn.cluster import KMeans
X = pd.read_csv("上海预处理后的数据归一化数据.csv")
estimator = KMeans(n_clusters = 3)#构造聚类器
estimator.fit(X)#聚类
label2 = estimator.labels_ #获取聚类标签
label2

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0])

In [43]:
a1, a2, a3 = [], [], []
for j in range(0,len(X)):
    if label2[j] == 2:
        a1.append(j)
    elif label2[j] == 1:
        a2.append(j)
    else:
        a3.append(j)
print(a1)
print(a2)
print(a3)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[20, 21, 22, 23, 24, 25, 26, 27, 28, 29]


In [10]:
## 三支聚类
data = X.drop(['Unnamed: 0'], axis = 1)
data.head()

,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,0.469165,0.593950,0.657676,0.636167,0.700524,0.764499,0.742700,0.805234,0.783095,0.575019,...,0.572985,0.700355,0.763832,0.572985,0.721611,0.486239,0.635739,0.657333,0.700781,0.635566
1,0.206467,0.221566,0.400280,0.453515,0.328810,0.292819,0.310316,0.472119,0.435467,0.293256,...,0.111441,0.364362,0.273312,0.210465,0.487696,0.115922,0.301130,0.316633,0.316372,0.348306
2,0.000000,0.378646,0.274077,0.412792,0.308878,0.377348,0.274722,0.377031,0.376406,0.409022,...,0.124394,0.565879,0.163120,0.234902,0.308878,0.120620,0.308878,0.232205,0.192351,0.347925
3,0.519631,0.548789,0.637934,0.637861,0.667310,0.637643,0.608195,0.696115,0.637359,0.579030,...,0.527512,0.608195,0.554106,0.499284,0.689969,0.333435,0.580372,0.663776,0.608195,0.552614
4,0.068102,0.520378,0.585417,0.674277,0.689968,0.532495,0.438979,0.625889,0.624414,0.359872,...,0.399335,0.417214,0.536810,0.351713,0.789891,0.299926,0.412641,0.482457,0.576495,0.451188


In [11]:
### 设定正理想解和负理想解
n, m = len(data), len(data.T)

### 计算决策对象到正负理想解的距离
D1 = np.zeros((1,n));D0 = np.zeros((1,n)); Pr = []
for i in range(0,n):
    D1[0][i] = np.sqrt(np.sum(np.square(1 - data.iloc[i]))) 
    D0[0][i] = np.sqrt(np.sum(np.square(data.iloc[i])))
    Pr.append(D0[0][i]/(D0[0][i] + D1[0][i]))
print(Pr)

[0.6608240483376462, 0.3847899290950833, 0.3151337132331966, 0.6096965020879236, 0.5072222039095763, 0.5150364521994567, 0.44042670270253464, 0.518646190560524, 0.41988894822949363, 0.39364153838113664, 0.4369686232275843, 0.5088258994387089, 0.5444224720335858, 0.511438612928836, 0.5140402866028393, 0.5208661297287724, 0.552233387913748, 0.5711790317902743, 0.37453817340123424, 0.36345352452424645, 0.4805400473651539, 0.3988505157049154, 0.4813259407351385, 0.4673401721190405, 0.5457673587337999, 0.6005954233165389, 0.3337029505244866, 0.6567996369497924, 0.5167800874003243, 0.49271430926533455]


In [12]:
def sunshi(x,singam):
    lamda = np.zeros((len(x),6));
    juhe = np.zeros((1,6));
    
    # 计算相对损失函数
    for i in range(0,len(x)):
        lamda[i,1] = singam[i] * x[i];
        lamda[i,2] = x[i];
        lamda[i,3] = 1-x[i];
        lamda[i,4] = singam[i] * (1-x[i]);
    
    # 计算聚合相对效用函数(均值聚合)
    juhe = np.mean(lamda,axis=0)
    
    return juhe

In [13]:
# 计算不同对象的聚合相对损失函数矩阵
singam = [0.45]*m
lamda = np.zeros((n,6));
for i in range(0,n):
    x = data.iloc[i]
    lamda[i] = sunshi(x ,singam)

#根据聚合相对损失函数计算不同对象的初始阈值
yuzhi = np.zeros((n,2))
for i in range(0,n):
    for j in range(0,len(lamda.T)):
        yuzhi[i][0] = (lamda[i][3]-lamda[i][4])/(lamda[i][3]-lamda[i][4]+lamda[i][1])
        yuzhi[i][1] = (lamda[i][4])/(lamda[i][4]+lamda[i][2]-lamda[i][1])
yuzhi = pd.DataFrame(yuzhi)
yuzhi.head()

,0,1
0,0.367207,0.279779
1,0.674166,0.580724
2,0.742386,0.658601
3,0.428706,0.334372
4,0.542116,0.442141


In [15]:
B1, B2, B3 = [], [], []
for i in range(n):
    if Pr[i] > yuzhi.loc[i][0]:
        B1.append(i)
    elif Pr[i]> yuzhi.loc[i][1] and Pr[i] < yuzhi.loc[i][0]:
        B2.append(i)
    else:
        B3.append(i)
print(B1)
print(B2)
print(B3)

[0, 3, 12, 16, 17, 24, 25, 27]
[4, 5, 7, 11, 13, 14, 15, 20, 22, 28, 29]
[1, 2, 6, 8, 9, 10, 18, 19, 21, 23, 26]


In [54]:
label3 = []
for i in range(0,30):
    if i in B1:
        label3.append(0)
    elif i in B2:
        label3.append(1)
    else:
        label3.append(2)
print(label3)

[0, 2, 2, 0, 1, 1, 2, 1, 2, 2, 2, 1, 0, 1, 1, 1, 0, 0, 2, 2, 1, 2, 1, 2, 0, 0, 2, 0, 1, 1]


In [17]:
## 基于遗憾理论的灰度关联聚类方法
r1 = np.zeros((n,m))
r2 = np.zeros((n,m))

for i in range(n):
    for j in range(m):
        r1[i][j] = 1/(1.5-data.loc[i][j])
        r2[i][j] = 1/(data.loc[i][j]+0.5)

In [18]:
# 建立感知效用矩阵
def Juzhen(alpha, sigma):
    U = np.zeros((n,m))
    R = r1/(r1+r2)
    G = 1 - np.exp(sigma * abs(r1-1))
    Q = 1 - np.exp(-sigma * abs(r1-1))
    for i in range(n):
        for j in range(m):
            U[i][j] = (R[i][j]) ** alpha + G[i][j] + Q[i][j] 
    return U

In [19]:
U = Juzhen(0.5,0.15)
U

array([[0.6960996 , 0.73933562, 0.76002552, ..., 0.75991693, 0.77342857,
        0.75296025],
       [0.59317595, 0.59958492, 0.67073989, ..., 0.63845634, 0.63835275,
        0.65087979],
       [0.49749948, 0.66255096, 0.62135999, ..., 0.60406001, 0.58712113,
        0.65073212],
       ...,
       [0.73468841, 0.75822358, 0.78915627, ..., 0.79305732, 0.76104121,
        0.78201635],
       [0.67132976, 0.70392117, 0.7072147 , ..., 0.71903398, 0.76404928,
        0.6137497 ],
       [0.69345099, 0.71177813, 0.74117943, ..., 0.73609092, 0.74164268,
        0.70962228]])

In [20]:
# 计算聚类系数
u = []
w = [0.1,0.2,0.5,0.05,0.15]
for i in range(n):
    u.append(np.sum(w * U[i][m-5:]))
print(u)

[0.7523992317002766, 0.6305552824780292, 0.6116062845289435, 0.7384568376038413, 0.6888017534230189, 0.7026149904146989, 0.6624051582716192, 0.7172166329714761, 0.6592716671579888, 0.646004178519296, 0.66559038948298, 0.6705380297598413, 0.7031160966380261, 0.6814598580622349, 0.7029487689849325, 0.7072152560052077, 0.7233350538006076, 0.7327679656143823, 0.6211905898600389, 0.6316912070212424, 0.6813330864392058, 0.6582664156144882, 0.6992109824638228, 0.6645863135374253, 0.7264839716187793, 0.7610265654738317, 0.6070145793161147, 0.7692306027585979, 0.6934082334679597, 0.7056463573241567]


In [21]:
C1, C2 = [], []
for i in range(n):
    if u[i] > np.mean(u):
        C1.append(i)
    else:
        C2.append(i)
print(C1)
print(C2)

[0, 3, 4, 5, 7, 12, 14, 15, 16, 17, 22, 24, 25, 27, 28, 29]
[1, 2, 6, 8, 9, 10, 11, 13, 18, 19, 20, 21, 23, 26]


In [39]:
from numpy import unique
from sklearn.cluster import AffinityPropagation
import numpy as np
from sklearn.preprocessing import StandardScaler

model = AffinityPropagation(damping=0.5, max_iter=200, convergence_iter=15, copy=True, preference=None,    
                            affinity='euclidean', verbose=False) # 设置damping : 阻尼系数，取值[0.5,1)
# 匹配模型
model.fit(X)
yhat = model.predict(X)  # yhat为集群结果
clusters = len(unique(yhat))  # 类别

print(yhat)

[0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 3 3 3 3 3 3]


In [40]:
D1, D2, D3, D4 = [], [], [], []
for i in range(n):
    if yhat[i] == 0:
        D1.append(i)
    if yhat[i] == 1:
        D2.append(i)
    if yhat[i] == 2:
        D3.append(i)
    if yhat[i] == 3:
        D4.append(i)
print(D1)
print(D2)
print(D3)
print(D4)

[0, 1, 2, 3, 4, 5, 6, 7, 8]
[9, 10, 11, 12, 13, 14, 15, 16, 17]
[18, 19, 20, 21, 22, 23]
[24, 25, 26, 27, 28, 29]


In [33]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler

In [53]:
# 计算Silhouette系数
silhouette_avg = silhouette_score(X, label2)
print(f"Silhouette Coefficient: {silhouette_avg}")

# 计算Davies-Bouldin指数
dbi = davies_bouldin_score(X, label2)
print(f"Davies-Bouldin Index: {dbi}")

Silhouette Coefficient: 0.4849001868969344
Davies-Bouldin Index: 0.6006407453793531


In [55]:
# 计算Silhouette系数
silhouette_avg = silhouette_score(X, label3)
print(f"Silhouette Coefficient: {silhouette_avg}")

# 计算Davies-Bouldin指数
dbi = davies_bouldin_score(X, label3)
print(f"Davies-Bouldin Index: {dbi}")

Silhouette Coefficient: -0.08379227007855476
Davies-Bouldin Index: 11.107586362014494
